In [1]:
import os
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import regex as re
import requests

pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment',None)

In [2]:
headers = {
    'authority': 'www.zillow.com',
    'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36',
    'accept': '*/*',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.zillow.com/los-angeles-ca/2_p/?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.88620504589845%2C%22east%22%3A-117.93726095410157%2C%22south%22%3A33.54554919445917%2C%22north%22%3A34.49481997322805%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D',
    'accept-language': 'en-US,en;q=0.9',
}


#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.get('https://www.zillow.com/search/GetSearchPageState.htm?searchQueryState=%7B%22pagination%22%3A%7B%22currentPage%22%3A2%7D%2C%22usersSearchTerm%22%3A%22Los%20Angeles%22%2C%22mapBounds%22%3A%7B%22west%22%3A-118.88620504589845%2C%22east%22%3A-117.93726095410157%2C%22south%22%3A33.54554919445917%2C%22north%22%3A34.49481997322805%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A12447%2C%22regionType%22%3A6%7D%5D%2C%22isMapVisible%22%3Atrue%2C%22filterState%22%3A%7B%22sortSelection%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22isAllHomes%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%7D&wants=\{%22cat1%22:\[%22listResults%22,%22mapResults%22\],%22cat2%22:\[%22total%22\]\}&requestId=3', headers=headers)


In [7]:
with requests.session() as s:
    city = 'los-angeles/'
    page = 1
    end_page = 10
    url = ''
    url_list = []
    
    while page <= end_page:
        url = 'https://www.zillow.com/homes/for_sale/' +city+ f'{page}_p/'
        url_list.append(url)
        page += 1
    
    request = ''
    request_list = []
    
    for url in url_list:
        request = s.get(url, headers=headers)
        request_list.append(request)
    
soup = ''
soup_list = []

for request in request_list:
    soup = BeautifulSoup(request.content, 'html.parser')
    soup_list.append(soup)

In [8]:
soup_list[0]

<!DOCTYPE html>
<html class="no-js zsg-theme-modernized null" itemscope="" itemtype="http://schema.org/Organization" lang="en" xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://ogp.me/ns#" xmlns:product="http://ogp.me/ns/product#">
<head>
<title>Los Angeles Real Estate - Los Angeles CA Homes For Sale | Zillow</title><meta content="Zillow has 6,016 homes for sale in Los Angeles CA. View listing photos, review sales history, and use our detailed real estate filters to find the perfect place." name="description"/><meta content="Zillow, Inc." name="author"/><meta content="Copyright (c) 2006-2021 Zillow, Inc." name="Copyright"/><script>(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
        (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
        m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
        })(window,document,'script','https://www.google-anal

In [9]:
df_list = []
for soup in soup_list:
    df = pd.DataFrame()
    for i in soup:
        address = soup.find_all (class_= 'list-card-addr')
        price = list(soup.find_all (class_='list-card-price'))
        beds = list(soup.find_all("ul", class_="list-card-details"))
        details = soup.find_all ('div', {'class': 'list-card-details'})
        home_type = soup.find_all ('div', {'class': 'list-card-footer'})
        last_updated = soup.find_all ('div', {'class': 'list-card-top'})
        brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
        link = soup.find_all (class_= 'list-card-link')
        
        df['prices'] = price
        df['address'] = address
        df['beds'] = beds
        
    urls = []
    
    for link in soup.find_all("article"):
        href = link.find('a',class_="list-card-link")
        addresses = href.find('address')
        addresses.extract()
        urls.append(href)
    
    df['links'] = urls
    df['links'] = df['links'].astype('str')
    df['links'] = df['links'].replace('<a class="list-card-link list-card-link-top-margin" href="', ' ', regex=True)
    df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)
    df_list.append(df)

In [11]:
df_list[0]

,prices,address,beds,links
0,"[$991,900]","[4929 Serrania Ave, Woodland Hills, CA 91364]","[[4, [ , , bds]], [4, [ , , ba]], [2,060, [ ...",https://www.zillow.com/homedetails/4929-Serra...
1,"[$414,900]","[13701 Hubbard St UNIT 22, Sylmar, CA 91342]","[[2, [ , , bds]], [2, [ , , ba]], [1,005, [ ...",https://www.zillow.com/homedetails/13701-Hubb...
2,"[$874,900]","[1135 S Shenandoah St UNIT 402, Los Angeles, C...","[[3, [ , , bds]], [3, [ , , ba]], [1,583, [ ...",https://www.zillow.com/homedetails/1135-S-She...
3,"[$619,900]","[11733 Avon Way APT 103, Los Angeles, CA 90066]","[[2, [ , , bds]], [2, [ , , ba]], [850, [ , ...",https://www.zillow.com/homedetails/11733-Avon...
4,"[$549,900]","[400 S La Fayette Park Pl APT 116, Los Angeles...","[[2, [ , , bds]], [2, [ , , ba]], [1,238, [ ...",https://www.zillow.com/homedetails/400-S-La-F...
5,"[$547,900]","[12955 Riverside Dr APT 209, Los Angeles, CA 9...","[[2, [ , , bds]], [2, [ , , ba]], [1,283, [ ...",https://www.zillow.com/homedetails/12955-Rive...
6,"[$534,900]","[740 N Kings Rd APT 319, West Hollywood, CA 90...","[[1, [ , , bd]], [1, [ , , ba]], [667, [ , ...",https://www.zillow.com/homedetails/740-N-King...
7,"[$180,000]","[13691 Gavina Ave UNIT 484, Sylmar, CA 91342]","[[3, [ , , bds]], [2, [ , , ba]], [1,534, [ ...",https://www.zillow.com/homedetails/13691-Gavi...
8,"[$725,000]","[1409 S Saltair Ave APT 103, Los Angeles, CA 9...","[[2, [ , , bds]], [2, [ , , ba]], [1,221, [ ...",https://www.zillow.com/homedetails/1409-S-Sal...
9,"[$349,000]","[21632 Blackhawk St, Chatsworth, CA 91311]","[[3, [ , , bds]], [2, [ , , ba]], [1,315, [ ...",https://www.zillow.com/homedetails/21632-Blac...


In [12]:
df = pd.concat(df_list).reset_index().drop(columns='index')

In [13]:
df

,prices,address,beds,links
0,"[$991,900]","[4929 Serrania Ave, Woodland Hills, CA 91364]","[[4, [ , , bds]], [4, [ , , ba]], [2,060, [ ...",https://www.zillow.com/homedetails/4929-Serra...
1,"[$414,900]","[13701 Hubbard St UNIT 22, Sylmar, CA 91342]","[[2, [ , , bds]], [2, [ , , ba]], [1,005, [ ...",https://www.zillow.com/homedetails/13701-Hubb...
2,"[$874,900]","[1135 S Shenandoah St UNIT 402, Los Angeles, C...","[[3, [ , , bds]], [3, [ , , ba]], [1,583, [ ...",https://www.zillow.com/homedetails/1135-S-She...
3,"[$619,900]","[11733 Avon Way APT 103, Los Angeles, CA 90066]","[[2, [ , , bds]], [2, [ , , ba]], [850, [ , ...",https://www.zillow.com/homedetails/11733-Avon...
4,"[$549,900]","[400 S La Fayette Park Pl APT 116, Los Angeles...","[[2, [ , , bds]], [2, [ , , ba]], [1,238, [ ...",https://www.zillow.com/homedetails/400-S-La-F...
5,"[$547,900]","[12955 Riverside Dr APT 209, Los Angeles, CA 9...","[[2, [ , , bds]], [2, [ , , ba]], [1,283, [ ...",https://www.zillow.com/homedetails/12955-Rive...
6,"[$534,900]","[740 N Kings Rd APT 319, West Hollywood, CA 90...","[[1, [ , , bd]], [1, [ , , ba]], [667, [ , ...",https://www.zillow.com/homedetails/740-N-King...
7,"[$180,000]","[13691 Gavina Ave UNIT 484, Sylmar, CA 91342]","[[3, [ , , bds]], [2, [ , , ba]], [1,534, [ ...",https://www.zillow.com/homedetails/13691-Gavi...
8,"[$725,000]","[1409 S Saltair Ave APT 103, Los Angeles, CA 9...","[[2, [ , , bds]], [2, [ , , ba]], [1,221, [ ...",https://www.zillow.com/homedetails/1409-S-Sal...
9,"[$349,000]","[21632 Blackhawk St, Chatsworth, CA 91311]","[[3, [ , , bds]], [2, [ , , ba]], [1,315, [ ...",https://www.zillow.com/homedetails/21632-Blac...


In [14]:
df['address'] = df['address'].astype('str')
df['beds'] = df['beds'].astype('str')

In [15]:
#remove html tags
#df['prices'] = df['prices'].replace('\[', '', regex=True)
df.loc[:,'address'] = df.loc[:,'address'].replace('<address class="list-card-addr">', '', regex=True)
#df['prices'] = df['prices'].replace('\]', '', regex=True)
df.loc[:,'address'] = df.loc[:,'address'].replace('</address>', '', regex=True)
#df['prices'] = df['prices'].str.replace(r'\D', '')

#filter unwanted property types
df = df[~df['beds'].str.contains("Land for sale")]

#remove html tags from beds column
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<ul class="list-card-details"><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bds</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->ba</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->bd</abbr></li><li class="">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!-- -->Foreclosure</abbr>', '- Foreclosure', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->sqft</abbr></li><li class="list-card-statusText">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('<abbr class="list-card-label"> <!-- -->acres lot</abbr></li><li class="list-card-statusText">', ' ', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('</li></ul>', '', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('--', '0', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('Multi-family', 'Multifamily', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace(' for sale', '', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Auction</abbr>', '- Auction', regex=True)
df.loc[:,'beds'] = df.loc[:,'beds'].replace('-<abbr class="list-card-label"> <!0 0>Pending</abbr>', '- Pending', regex=True)

#split beds column into beds, bath and sq_feet
#df[['beds','baths','sq_feet']] = df.beds.str.split(expand=True)

In [16]:
df.iloc[19]['beds']

' 5 8 15,642 - House'

In [17]:
df[['beds','type']] = df.beds.apply(
    lambda x: pd.Series(str(x).split('-')))

In [18]:
df.head()

,prices,address,beds,links,type
0,"[$991,900]","4929 Serrania Ave, Woodland Hills, CA 91364","4 4 2,060",https://www.zillow.com/homedetails/4929-Serra...,House
1,"[$414,900]","13701 Hubbard St UNIT 22, Sylmar, CA 91342","2 2 1,005",https://www.zillow.com/homedetails/13701-Hubb...,Townhouse
2,"[$874,900]","1135 S Shenandoah St UNIT 402, Los Angeles, CA...","3 3 1,583",https://www.zillow.com/homedetails/1135-S-She...,Condo
3,"[$619,900]","11733 Avon Way APT 103, Los Angeles, CA 90066",2 2 850,https://www.zillow.com/homedetails/11733-Avon...,Condo
4,"[$549,900]","400 S La Fayette Park Pl APT 116, Los Angeles,...","2 2 1,238",https://www.zillow.com/homedetails/400-S-La-F...,Condo


In [19]:
df[['beds', 'baths', 'sq_feet']] = df.beds.str.split(expand=True)

In [20]:
df

,prices,address,beds,links,type,baths,sq_feet
0,"[$991,900]","4929 Serrania Ave, Woodland Hills, CA 91364",4,https://www.zillow.com/homedetails/4929-Serra...,House,4,"2,060"
1,"[$414,900]","13701 Hubbard St UNIT 22, Sylmar, CA 91342",2,https://www.zillow.com/homedetails/13701-Hubb...,Townhouse,2,"1,005"
2,"[$874,900]","1135 S Shenandoah St UNIT 402, Los Angeles, CA...",3,https://www.zillow.com/homedetails/1135-S-She...,Condo,3,"1,583"
3,"[$619,900]","11733 Avon Way APT 103, Los Angeles, CA 90066",2,https://www.zillow.com/homedetails/11733-Avon...,Condo,2,850
4,"[$549,900]","400 S La Fayette Park Pl APT 116, Los Angeles,...",2,https://www.zillow.com/homedetails/400-S-La-F...,Condo,2,"1,238"
5,"[$547,900]","12955 Riverside Dr APT 209, Los Angeles, CA 91423",2,https://www.zillow.com/homedetails/12955-Rive...,Condo,2,"1,283"
6,"[$534,900]","740 N Kings Rd APT 319, West Hollywood, CA 90069",1,https://www.zillow.com/homedetails/740-N-King...,Condo,1,667
7,"[$180,000]","13691 Gavina Ave UNIT 484, Sylmar, CA 91342",3,https://www.zillow.com/homedetails/13691-Gavi...,Home,2,"1,534"
8,"[$725,000]","1409 S Saltair Ave APT 103, Los Angeles, CA 90025",2,https://www.zillow.com/homedetails/1409-S-Sal...,Condo,2,"1,221"
9,"[$349,000]","21632 Blackhawk St, Chatsworth, CA 91311",3,https://www.zillow.com/homedetails/21632-Blac...,House,2,"1,315"
